In [1]:
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2020-06-06 07:46:58--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 143.204.160.55, 143.204.160.6, 143.204.160.108, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|143.204.160.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘chr1.GRCh38.excerpt.fasta.1’

chr1.GRCh38.excerpt 100%[===================>] 791.12K  4.63MB/s    in 0.2s    

2020-06-06 07:46:59 (4.63 MB/s) - ‘chr1.GRCh38.excerpt.fasta.1’ saved [810105/810105]



In [104]:
# read a genome
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

# parses fastq files
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [2]:
chr1 = readGenome("chr1.GRCh38.excerpt.fasta.1")

In [23]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = 0
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return min(D[-1][:])

In [29]:
editDistance("GCTGATCGATCGTACG",chr1)

3

In [30]:
editDistance('GATTTACCAGATTGAG',chr1)

2

In [111]:
from itertools import permutations,combinations
def overlap(a, b, min_length):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            pass
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return (a,b)
        start += 1  # move just past previous match

In [112]:
reads = ['CGTACG', 'TACGTA', 'GTACGT', 'ACGTAC', 'GTACGA', 'TACGAT']
def overlap(reads, min_length):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    collect = []
    start = 0  # start all the way at the left
    for combo in permutations(reads,2):
        start = 0
        a = combo[0]
        b = combo[1]
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            continue
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            collect.append((a,b))
    
        start += 1  # move just past previous match
    return collect
result = overlap(reads,4)
print(result)

[('CGTACG', 'TACGTA'), ('CGTACG', 'GTACGT'), ('CGTACG', 'GTACGA'), ('CGTACG', 'TACGAT'), ('TACGTA', 'CGTACG'), ('TACGTA', 'ACGTAC'), ('GTACGT', 'TACGTA'), ('GTACGT', 'ACGTAC'), ('ACGTAC', 'CGTACG'), ('ACGTAC', 'GTACGT'), ('ACGTAC', 'GTACGA'), ('GTACGA', 'TACGAT')]


In [119]:
reads = ['CGTACG', 'TACGTA', 'GTACGT', 'ACGTAC', 'GTACGA', 'TACGAT']
def overlap_for_reads(reads, min_length):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    huge_dict = dict()

    for read in reads:
        for number in range(0,len(read)+1):
            if len(read[number:number+min_length]) == min_length:
                if read[number:number+min_length] in huge_dict.keys():
                    huge_dict[read[number:number+min_length]].add(read)
                if not read[number:number+min_length] in huge_dict.keys():
                    reads_set = set()
                    reads_set.add(read)
                    huge_dict[read[number:number+min_length]] = reads_set
    collect = set()
    for kmer,collected_reads in huge_dict.items():
        if len(reads) > 2:
#             collect = []
            for read in permutations(collected_reads,2):
                start = 0
                a = read[0]
                b = read[1]
#                 print(a,b)
                start = a.find(b[:min_length], start)  # look for b's prefix in a
#                 print(start)
                if start == -1:  # no more occurrences to right
                    continue
                # found occurrence; check for full suffix/prefix match
                if b.startswith(a[start:]):
                    collect.add((a,b))

                start += 1  # move just past previous match
    return collect
result = overlap_for_reads(reads,5)
print(result)

{('ACGTAC', 'CGTACG'), ('GTACGT', 'TACGTA'), ('CGTACG', 'GTACGA'), ('TACGTA', 'ACGTAC'), ('GTACGA', 'TACGAT'), ('CGTACG', 'GTACGT')}


In [42]:
reads = ['CGTACG', 'TACGTA', 'GTACGT', 'ACGTAC', 'GTACGA', 'TACGAT']
huge_dict = {}

for read in reads:
    reads_set = []
    for number in range(0,len(read)+1):
        if len(read[number:number+5]) == 5:
            if read[number:number+5] in huge_dict.keys():
                if not read in huge_dict[read[number:number+5]]:
                    huge_dict[read[number:number+5]].append(read)
            else:
                huge_dict[read[number:number+5]] = [read]
from itertools import permutations
x = []
for key in huge_dict.keys():
#     print(key)
    if len(huge_dict[key]) >= 2:
        for value in set(permutations(huge_dict[key],2)):
            x.append(overlap(value[0],value[1],5))
print(x)
print(set(x))
print(len(set(x))-1)

KeyboardInterrupt: 

In [74]:
!wget https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq

--2020-06-06 08:39:41--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 143.204.160.6, 143.204.160.98, 143.204.160.108, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|143.204.160.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2562951 (2.4M) [application/octet-stream]
Saving to: ‘ERR266411_1.for_asm.fastq’

ERR266411_1.for_asm 100%[===================>]   2.44M  1.74MB/s    in 1.4s    

2020-06-06 08:39:43 (1.74 MB/s) - ‘ERR266411_1.for_asm.fastq’ saved [2562951/2562951]



In [123]:
reads, qualitites = readFastq('ERR266411_1.for_asm.fastq')

In [124]:
len(reads)

10000

In [125]:
def overlap_for_reads(reads, min_length):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    huge_dict = dict()

    for read in reads:
        for number in range(0,len(read)+1):
            if len(read[number:number+min_length]) == min_length:
                if read[number:number+min_length] in huge_dict.keys():
                    huge_dict[read[number:number+min_length]].add(read)
                if not read[number:number+min_length] in huge_dict.keys():
                    reads_set = set()
                    reads_set.add(read)
                    huge_dict[read[number:number+min_length]] = reads_set
    collect = set()
    for kmer,collected_reads in huge_dict.items():
        if len(collected_reads) >= 2:
#             collect = []
            for read in permutations(collected_reads,2):
                start = 0
                a = read[0]
                b = read[1]
#                 print(a,b)
                start = a.find(b[:min_length], start)  # look for b's prefix in a
#                 print(start)
                if start == -1:  # no more occurrences to right
                    continue
                # found occurrence; check for full suffix/prefix match
                if b.startswith(a[start:]):
                    collect.add((a,b))

                start += 1  # move just past previous match
    return collect
result = overlap_for_reads(reads,30)
print(result)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [160]:
%%time
reads, qualitites = readFastq('ERR266411_1.for_asm.fastq')
# reads = ['CGTACG', 'TACGTA', 'GTACGT', 'ACGTAC', 'GTACGA', 'TACGAT']

from collections import defaultdict
def overlap_hw(a, b, min_length):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match
def overlap_for_reads(reads, min_length):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    huge_dict = defaultdict(set)

    for read in reads:
        for number in range(0,len(read)+1):
            if len(read[number:number+min_length]) == min_length:
                huge_dict[read[number:number+min_length]].add(read)

    print(len(huge_dict.keys()))
    collect = set()
    collect_read_path = dict()
    for read in reads:
        if len(huge_dict[read[len(read)-min_length:len(read)+1]]) >= 2:
            for specific_read in huge_dict[read[len(read)-min_length:len(read)]]:
                if not read == specific_read:
#                     overlap_hw(read,specific_read,min_length)
                    x = overlap_hw(read,specific_read,min_length)
                    if x != 0:
                        collect.add((read,specific_read))
                        collect_read_path[read] = specific_read
                        

    return collect,collect_read_path
question3, question4 = overlap_for_reads(reads,30)
print(len(question3))
print(len(question4))

108344
904746
7161
CPU times: user 5.23 s, sys: 167 ms, total: 5.4 s
Wall time: 5.5 s
